## Build Your Own Code Interpreter Tool for LLM Agents 

Many API providers—such as OpenAI’s Assistants API—offer built-in code interpreter functionality. These built-in tools can be immensely powerful, but there are situations where developers may need to create their own custom code interpreter. For example:

1. **Language or library support**: The built-in interpreter may not support the specific programming language or libraries required for your task.
2. **Task compatibility**: Your use case may not be compatible with the provider’s built-in solution.
3. **Model constraints**: You might require a language model that isn’t supported by the provider’s interpreter.
4. **Cost considerations**: The cost structure for code execution or model usage may not fit your budget or constraints.  
5. **File size**: The file size of input data is too large or not supported by the provider's interpreter.

At the core of this approach is “function (or tool) calling,” where a Large Language Model (LLM) can invoke a function with arguments. Typically, these functions are predefined by the developer, along with their expected arguments and outputs. However, in this Cookbook, we explore a more flexible paradigm.

### Dynamically Generated Tool Calling with Code Interpreter
A Dynamically Generated Tool is a function or code block created by the LLM itself at runtime based on the user’s prompt. This means you don’t have to predefine every possible scenario in your codebase—enabling far more open-ended, creative, and adaptive problem-solving.

Dynamically Generated Tool Calling goes a step further by granting the LLM the ability to generate tools and execute code blocks on the fly. This dynamic approach is particularly useful for tasks that involve:

- Data analysis and visualization
- Data manipulation and transformation
- Machine learning workflow generation and execution
- Process automation and scripting
- And much more, as new possibilities emerge through experimentation

### What You’ll Learn
By following this Cookbook, you will learn how to:

- Set up an isolated Python code execution environment using Docker
- Configure your own code interpreter tool for LLM agents
- Establish a clear separation of “Agentic” concerns for security and safety
- Orchestrate agents to efficiently accomplish a given task
- Design an agentic application that can dynamically generate and execute code

You’ll see how to build a custom code interpreter tool from the ground up, leverage the power of LLMs to generate sophisticated code, and safely execute that code in an isolated environment—all in pursuit of making your AI-powered applications more flexible, powerful, and cost-effective.

### Example Scenario

We'll use the sample data in ![Key Factors Traffic Accidents](https://www.kaggle.com/datasets/willianoliveiragibin/key-factors-traffic-accidents) to answer a set of questions. These questions do not require to be pre-defined, we will give LLM the ability to generate code to answer any question. 

Sample questions could be: 
- What factors contribute the most to accident frequency? (Feature importance analysis)
- Which areas are at the highest risk of accidents? (Classification/Clustering)
- How does traffic fine amount influence the number of accidents? (Regression/Causal inference)
- Can we determine the optimal fine amounts to reduce accident rates? (Optimization models)
- Do higher fines correlate with lower average speeds or reduced accidents? (Correlation/Regression)
- and so on ...

Using the traditional **Predefined Tool Calling** approach, we would need to pre-define the function for each of these questions. This limits the LLM's ability to answer any other questions not defined in the pre-defined set of functions. We overcome this limitation by using the **Dynamic Tool Calling** approach where the LLM generates code and uses a Code Interpretter tool to execute the code. 

## Overview
Let's dive into the steps to build this Agentic Applicaiton with Dynamic Tool Calling. There are three components to this application:

#### Step 1: Set up an isolated code execution container environment

We need a secure environment where our LLM generated function calls can be executed. We want to avoid directly running the LLM generated code on the host machine so will create a Docker container environment with restricted resource access (e.g., no network access). By default, Docker containers cannot access the host machine’s file system, which helps ensure that any code generated by the LLM remains contained. 

##### ⚠️ A WORD OF CAUTION:        
LLMs could generate harmful code with unintended consequences. As a best practice, isolate the code execution environment with only required access to resources as needed by the task. Avoid running the LLM generated code on your host machine or laptop. 

#### Step 2: Define and Test the Agents

"**What is an Agent?**" In the context of this Cookbook, an Agent is:
1. Set of instructions for the LLM to follow, i.e. the developer prompt
2. A LLM model, and ability to call the model via the API 
3. Tool call access to a function, and ability to call the function 

We will define two agents:
1. FileAccessAgent: This agent will read the file and provide the context to the PythonCodeExecAgent.
2. PythonCodeExecAgent: This agent will generate the Python code to answer the user's question and execute the code in the Docker container.

#### Step 3: Set up Agentic Orchestration to run the application 
There are various ways to orchestrate the Agents based on the application requirements. In this example, we will use a simple orchestration where the user provides a task and the agents are called in sequence to accomplish the task.  

## Let's get started


### Prerequisites
Before you begin, ensure you have the following installed and configured on your host machine:

1. Docker: installed and running on your local machine. You can learn more about Docker and [install it from here](https://www.docker.com/). 
2. Python: installed on your local machine. You can learn more about Python and [install it from here](https://www.python.org/downloads/). 
3. OpenAI API key: set up on your local machine as an environment variable. You can learn more about OpenAI API key and [set it up from here](https://platform.openai.com/docs/api-reference/introduction). 


### Step 1: Set up an Isolated Code Execution Environment 

Lets define a Dockerized container environment that will be used to execute our code. I have defined the dockerfile in the directory `./resources/docker/dockerfile` that will be used to create the container environment with the following specifications:
- Python 3.10 as the base 
- A non-root user 
- Preinstall the packages in requirements.txt  

The requirements.txt included in the docker image creation process contains all the potential packages our LLM generated code may need to accomplish its tasks. Given we will restrict the container from network access, so we need to pre-install the packages that are required for the task. Our LLM will not be allowed to install any additional packages for security purposes. 

You could create your own docker image with the language requirements (such as Python 3.10) and pre-install the packages that are required for the task, or create a custom docker image with the specific language (such as Java, C++, etc.) and packages that are required for the task. 

Let's build the docker image with the following command. For the sake of brevity, I have redirected the output to grep the success message and print a message if the build fails.

In [ ]:
!docker build -t python_sandbox:latest ./resources/docker 2>&1 | grep -E "View build details|ERROR" || echo "Build failed."

Let's run the container in restricted mode. The container will run in the background. This is our opportunity to define the security policies for the container. It is good practice to only allow the bare minimum features to the container that are required for the task. By default, the container cannot access the host file system from within the container. Let's also restrict its access to network so it cannot access the Internet or any other network resources. 

In [ ]:
# Run the container in restricted mode. The container will run in the background.
!docker run -d --name sandbox --network none --cap-drop all --pids-limit 64 --tmpfs /tmp:rw,size=64M   python_sandbox:latest sleep infinity

Let's make sure container is running using the `docker ps` that should list our container. 

In [1]:
!docker ps 

CONTAINER ID   IMAGE                   COMMAND            CREATED       STATUS       PORTS     NAMES
18bb7f906b2d   python_sandbox:latest   "sleep infinity"   2 hours ago   Up 2 hours             sandbox


### Step 2: Define and Test the Agents

For our purposes, we will define two agents. 
1.	**Agent 1: File Access Agent (with Pre-defined Tool Calling)**
- Instructions to understand the contents of the file to provide as context to Agent 2.
- Has access to the host machine’s file system. 
- Can read a file from the host and copy it into the Docker container.
- Cannot access the code interpreter tool. 

2.	**Agent 2: Python Code Generator and Executor (with Dynamically Generated Tool Calling and Code Execution)**
- Recieve the file content's context from Agent 1.
- Instructions to generate a Python script to answer the user's question.
- Has access to the code interpreter within the Docker container, which is used to execute Python code.
- Has access only to the file system inside the Docker container (not the host).
- Cannot access the host machine’s file system or the network.

This separation concerns of the File Access (Agent 1) and the Code Generator and Executor (Agent 2) is crucial to prevent the LLM from directly accessing or modifying the host machine. 


**Limit the Agent 1 to Static Tool Calling as it has access to the host file system.**  


| Agent | Type of Tool Call | Access to Host File System | Access to Docker Container File System | Access to Code Interpreter |
|-------|-------------------|----------------------------|----------------------------------------|----------------------------|
| Agent 1: File Access | Pre-defined Tools | Yes | Yes | No |
| Agent 2: Python Code Generator and Executor | Dynamically Generated Tools | No | Yes | Yes |


To keep the Agents and Tools organized, we've defined a set of core classes that will be used to create the two agents for consistency using Object Oriented Programming principles.

- **BaseAgent**: We start with an abstract base class that enforces a task method signature. This ensures that all concrete agents will implement task consistently.
- **ChatMessages**: A class to store the conversation history given ChatCompletions API is stateless
- **ToolManager**: A class to manage the tools that an agent can call.
- **ToolInterface**: An abstract class for any 'tool' that an agent can call so that the tools will have a consistent interface.

These classes are defined in the `./resources/object_oriented_agents/core_classes` directory. Let's add the parent directory to the Python path so we can import these classes in our notebook. 

In [2]:
import sys, os

# Add the parent directory of 'core_classes' to the Python path
sys.path.append(os.path.abspath('resources/object_oriented_agents/core_classes'))

**Define Agent 1: FileAccessTool and FileAccessAgent**

Let's start with definin the FileAccessTool that inherits from the ToolInterface class. Inheriting from the ToolInterface class ensures that the tools will have a consistent interface. Binding the tool definition (for the OpenAI Function Calling API) to the tool's run method ensure maintainability, scalability, and reusability. 

In [3]:
from typing import Dict, Any
import pandas as pd
import subprocess
import os
import logging 

from resources.object_oriented_agents.core_classes.tool_interface import ToolInterface

class FileAccessTool(ToolInterface):
    """
    A tool to read CSV files and copy them to a Docker container.
    """

    def __init__(self, logger=None):
        self.logger = logger or get_logger(self.__class__.__name__)

    def get_definition(self) -> Dict[str, Any]:
        self.logger.debug("Returning tool definition for safe_file_access")
        return {
            "function": {
                "name": "safe_file_access",
                "description": (
                    "Read the contents of a file in a secure manner "
                    "and transfer it to the Python code interpreter docker container"
                ),
                "parameters": {
                    "type": "object",
                    "properties": {
                        "filename": {
                            "type": "string",
                            "description": "Name of the file to read"
                        }
                    },
                    "required": ["filename"]
                }
            }
        }

    def run(self, arguments: Dict[str, Any]) -> str:
        filename = arguments["filename"]
        self.logger.debug(f"Running safe_file_access with filename: {filename}")
        
        return self.safe_file_access(filename)

    def safe_file_access(self, filename: str) -> str:
        if not filename.endswith('.csv'):
            error_msg = "Error: The file is not a CSV file."
            self.logger.warning(f"{error_msg} - Filename provided: {filename}")
            return error_msg

        # Ensure the path is correct
        if not os.path.dirname(filename):
        
            filename = os.path.join('./resources/data', filename)
        
        self.logger.debug(f"Attempting to read file at path: {filename}")
        try:
            df = pd.read_csv(filename)
            self.logger.debug(f"File '{filename}' loaded successfully.")
            copy_output = self.copy_file_to_container(filename)
            head_str = df.head(15).to_string()
            return f"{copy_output}\nThe file content for the first 15 rows is:\n{head_str}"
        except FileNotFoundError:
            error_msg = f"Error: The file '{filename}' was not found."
            self.logger.error(error_msg)
            return error_msg
        except Exception as e:
            error_msg = f"Error while reading the CSV file: {str(e)}"
            self.logger.error(error_msg, exc_info=True)
            return error_msg

    def copy_file_to_container(self, local_file_name: str, container_name: str = "sandbox") -> str:
        container_home_path = "/home/sandboxuser"
        self.logger.debug(f"Copying '{local_file_name}' to container '{container_name}'.")

        if not os.path.isfile(local_file_name):
            error_msg = f"The local file '{local_file_name}' does not exist."
            self.logger.error(error_msg)
            raise FileNotFoundError(error_msg)

        # Check if container is running
        check_container_cmd = ["docker", "inspect", "-f", "{{.State.Running}}", container_name]
        result = subprocess.run(check_container_cmd, capture_output=True, text=True)
        if result.returncode != 0 or result.stdout.strip() != "true":
            error_msg = f"The container '{container_name}' is not running."
            self.logger.error(error_msg)
            raise RuntimeError(error_msg)

        # Copy the file into the container
        container_path = f"{container_name}:{container_home_path}/{os.path.basename(local_file_name)}"
        self.logger.debug(f"Running command: docker cp {local_file_name} {container_path}")
        subprocess.run(["docker", "cp", local_file_name, container_path], check=True)

        # Verify the file was copied
        verify_cmd = ["docker", "exec", container_name, "test", "-f",
                      f"{container_home_path}/{os.path.basename(local_file_name)}"]
        verify_result = subprocess.run(verify_cmd, capture_output=True, text=True)
        if verify_result.returncode != 0:
            error_msg = f"Failed to verify the file '{local_file_name}' in the container '{container_name}'."
            self.logger.error(error_msg)
            raise RuntimeError(error_msg)

        success_msg = f"Copied {local_file_name} into {container_name}:{container_home_path}/."
        self.logger.debug(success_msg)
        return success_msg

Now, let's define the FileAccessAgent that extends the BaseAgent class and bind the FileAccessTool to the agent. 

In [6]:
from resources.object_oriented_agents.utils.logger import get_logger
from resources.object_oriented_agents.core_classes.base_agent import BaseAgent
from resources.object_oriented_agents.core_classes.tool_manager import ToolManager
from resources.object_oriented_agents.services.openai_language_model import OpenAILanguageModel


# Set the verbosity level: DEBUG for verbose output, INFO for normal output, and WARNING/ERROR for minimal output
myapp_logger = get_logger("MyApp", level=logging.INFO)

# Create a LanguageModelInterface instance using the OpenAILanguageModel
language_model_api_interface = OpenAILanguageModel(api_key=os.getenv("OPENAI_API_KEY"), logger=myapp_logger)


class FileAccessAgent(BaseAgent):
    """
    Agent that can only use the 'safe_file_access' tool to read CSV files.
    """
    # We pass the Agent attributes in the constructor 
    def __init__(self, 
                 developer_prompt: str = """
                 You are a helpful data science assistant. The user will provide the name of a CSV file that contains relational data. The file is in the directory ./resources/data

                 Instructions:
                 1. When the user provides the CSV file name, use the 'safe_read_file' tool to read and display the first 15 lines of that file.
                 2. If the specified file does not exist in the provided directory, return an appropriate error message (e.g., "Error: The specified file does not exist in the provided directory").
                 3. The user may request data analysis based on the file’s contents, but you should NOT perform or write code for any data analysis. Your only task is to read and return the first 6 lines of the file.

                 Do not include any additional commentary or code not related to reading the file.
                 """,
                 model_name: str = "gpt-4o",
                 language_model_interface = language_model_api_interface,
                 logger = myapp_logger):
        super().__init__(developer_prompt=developer_prompt, model_name=model_name, logger=logger, language_model_interface=language_model_interface)
        self.setup_tools()

    def setup_tools(self) -> None:
        self.logger.debug("Setting up tools for FileAccessAgent.")
        # Pass the openai_client to ToolManager
        self.tool_manager = ToolManager(logger=self.logger, language_model_interface=self.language_model_interface)
        # Register the one tool this agent is allowed to use
        self.tool_manager.register_tool(FileAccessTool(logger=self.logger))

**Define Agent 2: PythonExecTool and PythonExecAgent**  

Similarly, PythonExecTool inherits from the ToolInterface class and implements the get_definition and run methods. The get_definition method returns the tool definition in the format expected by the OpenAI Function Calling API. The run method executes the Python code in a Docker container and returns the output.

In [7]:
from typing import Tuple, Optional

class PythonExecTool(ToolInterface):
    """
    A Tool that executes Python code securely in a container.
    """

    def get_definition(self) -> Dict[str, Any]:
        """
        Return the JSON/dict definition of the tool's function
        in the format expected by the OpenAI function calling API.
        """
        return {
            "function": {
                "name": "execute_python_code",
                "description": "Executes Python code securely in a container. Python version 3.10 is installed in the container. pandas, numpy, matplotlib, seaborn, and scikit-learn are installed in the container.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "python_code": {
                            "type": "string",
                            "description": "The Python code to execute"
                        }
                    },
                    "required": ["python_code"]
                }
            }
        }

    def run(self, arguments: Dict[str, Any]) -> str:
        """
        Execute the Python code in a Docker container and return the output.
        """
        python_code = arguments["python_code"]
        python_code_stripped = python_code.strip('"""')

        output, errors = self._run_code_in_container(python_code_stripped)
        if errors:
            return f"[Error]\n{errors}"

        return output

    @staticmethod
    def _run_code_in_container(code: str, container_name: str = "sandbox") -> Tuple[str, str]:
        """
        Helper function that actually runs Python code inside a Docker container named `sandbox` (by default).
        """
        cmd = [
            "docker", "exec", "-i",
            container_name,
            "python", "-c", "import sys; exec(sys.stdin.read())"
        ]

        process = subprocess.Popen(
            cmd,
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
        out, err = process.communicate(code)
        return out, err

Now, let's define the PythonExecAgent agent that will uses this tool. 

In [8]:
class PythonExecAgent(BaseAgent):
    """
    An agent specialized in executing Python code in a Docker container.
    """

    def __init__(
            self,
            developer_prompt: str = """  
                    You are a helpful data science assistant. Your tasks include analyzing CSV data and generating Python code to address user queries. Follow these guidelines:

                    1. The user will provide the name of a CSV file located in the directory `/home/sandboxuser`.
                    2. The user will also supply context, including:
                    - Column names and their descriptions.
                    - Sample data from the CSV (headers and a few rows) to help understand data types.
                    3. Analyze the provided data using Python machine learning libraries and generate appropriate code to fulfill the user's request.
                    4. Generate Python code to analyze the data and call the tool `execute_python_code` to run the code inside a Docker container.
                    5. Execute the code in the container and return the output.
                    
                    Note: All files referenced in the prompt are located in `/home/sandboxuser`.
                """,
            model_name: str = "o1",
            logger=myapp_logger,
            language_model_interface = language_model_api_interface
        ):
        super().__init__(
            developer_prompt=developer_prompt,
            model_name=model_name,
            logger=logger,
            language_model_interface=language_model_interface
        )
        self.setup_tools()

    def setup_tools(self) -> None:
        """
        Create a ToolManager, instantiate the PythonExecTool and register it with the ToolManager.
        """
        self.tool_manager = ToolManager(logger=self.logger, language_model_interface=self.language_model_interface)
        
        # Create the Python execution tool
        python_exec_tool = PythonExecTool()
        
        # Register the Python execution tool
        self.tool_manager.register_tool(python_exec_tool)

### Step 3: Set up Agentic Orchestration to run the application 

Let's define the orchestration loop that will run the application. This loop will prompt the user for a question or task, and then call the FileAccessAgent to read the file and provide the context to the PythonExecAgent. The PythonExecAgent will generate the Python code to answer the user's question and execute the code in the Docker container. The output from the code execution will be displayed to the user. 

User can type 'exit' to stop the application. 

The overall orchestration is shown below:

![Agentic Workflow Orchestration](./resources/images/AgenticWorkflow.png)

In [9]:
import base64
import binascii


prompt = """Use the file traffic_accidents.csv for your analysis. The column names are:
Variable	Description
accidents	Number of recorded accidents, as a positive integer.
traffic_fine_amount	Traffic fine amount, expressed in thousands of USD.
traffic_density	Traffic density index, scale from 0 (low) to 10 (high).
traffic_lights	Proportion of traffic lights in the area (0 to 1).
pavement_quality	Pavement quality, scale from 0 (very poor) to 5 (excellent).
urban_area	Urban area (1) or rural area (0), as an integer.
average_speed	Average speed of vehicles in km/h.
rain_intensity	Rain intensity, scale from 0 (no rain) to 3 (heavy rain).
vehicle_count	Estimated number of vehicles, in thousands, as an integer.
time_of_day	Time of day in 24-hour format (0 to 24).
accidents	traffic_fine_amount
"""


print("Setup: ")
print(prompt)

print("Setting up the agents... ")
# instantiate the agents 
file_ingestion_agent = FileAccessAgent()
data_analysis_agent = PythonExecAgent()

file_ingestion_agent_output = file_ingestion_agent.task(prompt) 



print("Understanding the contents of the file...")

# add the file content as context to the data analysis agent 
data_analysis_agent.add_context(prompt)
data_analysis_agent.add_context(file_ingestion_agent_output)

while True:

    print("Type your question related to the data in the file. Type 'exit' to exit.")
    user_input = input("Type your question.")

    if user_input == "exit":
        print("Exiting the application.")
        break

    print("Generating dynamic tools and using code interpreter...")
    data_analysis_agent_output = data_analysis_agent.task(user_input)

    print("Output...")
    print(data_analysis_agent_output)



Setup: 
Use the file traffic_accidents.csv for your analysis. The column names are:
Variable	Description
accidents	Number of recorded accidents, as a positive integer.
traffic_fine_amount	Traffic fine amount, expressed in thousands of USD.
traffic_density	Traffic density index, scale from 0 (low) to 10 (high).
traffic_lights	Proportion of traffic lights in the area (0 to 1).
pavement_quality	Pavement quality, scale from 0 (very poor) to 5 (excellent).
urban_area	Urban area (1) or rural area (0), as an integer.
average_speed	Average speed of vehicles in km/h.
rain_intensity	Rain intensity, scale from 0 (no rain) to 3 (heavy rain).
vehicle_count	Estimated number of vehicles, in thousands, as an integer.
time_of_day	Time of day in 24-hour format (0 to 24).
accidents	traffic_fine_amount

Setting up the agents... 


2025-01-21 13:15:50,793 - MyApp - INFO - Handling tool call: safe_file_access
2025-01-21 13:15:50,794 - MyApp - INFO - Tool arguments: {'filename': './resources/data/traffic_accidents.csv'}
2025-01-21 13:15:50,985 - MyApp - INFO - Tool 'safe_file_access' response: Copied ./resources/data/traffic_accidents.csv into sandbox:/home/sandboxuser/.
The file content for the first 15 rows is:
    accidents  traffic_fine_amount  traffic_density  traffic_lights  pavement_quality  urban_area  average_speed  rain_intensity  vehicle_count  time_of_day
0          20               4.3709           2.3049         753.000            0.7700           1        321.592          1.1944       290.8570     160.4320
1          11               9.5564           3.2757           5.452            4.0540           1        478.623          6.2960       931.8120       8.9108
2          19               7.5879           2.0989           6.697          345.0000           0        364.476          2.8584       830.086

Understanding the contents of the file...
Type your question related to the data in the file. Type 'exit' to exit.
Generating dynamic tools and using code interpreter...


2025-01-21 13:16:27,227 - MyApp - INFO - Handling tool call: execute_python_code
2025-01-21 13:16:27,229 - MyApp - INFO - Tool arguments: {'python_code': 'import pandas as pd\nimport numpy as np\nfrom sklearn.ensemble import RandomForestRegressor\nimport matplotlib.pyplot as plt\nimport seaborn as sns\n\n# 1. Load the dataset\nfile_path = \'/home/sandboxuser/traffic_accidents.csv\'\ndata = pd.read_csv(file_path)\n\n# 2. Display basic information\nprint(\'Data preview:\')\nprint(data.head())\nprint("\\nData description:\\n", data.describe())\n\n# 3. Calculate correlation matrix to see how \'traffic_lights\' correlates with \'accidents\'\ncorrelation_matrix = data.corr()\nprint("\\nCorrelation matrix:\\n", correlation_matrix)\nprint("\\nCorrelation of \'traffic_lights\' with \'accidents\':", correlation_matrix.loc[\'accidents\',\'traffic_lights\'])\n\n# 4. Prepare data for modeling\n#    We\'ll use accidents as the target, and all other numeric columns as features.\n\n# Drop non-numeric 

Output...
Below is a summary of the findings regarding the role of traffic lights (“traffic_lights”) in preventing accidents, based on the analysis:

1. Correlation with Accidents:
   • The correlation coefficient between traffic_lights and accidents is approximately –0.0266. This is a small negative correlation, indicating that higher proportions of traffic lights are (very) slightly associated with fewer accidents. However, because this value is close to zero, it suggests that in this dataset the relationship between the two variables is weak.

2. Feature Importance from Random Forest:
   • A Random Forest Regressor was trained to predict accidents using all other columns. The relative importance of traffic_lights for predicting accidents is about 0.0226 (2.26%), which is low compared with traffic_fine_amount (≈58.08%) or traffic_density (≈16.52%). 
   • Thus, in this model, traffic_lights does not appear to be a major factor compared to other variables when predicting accident count

In this example, the LLM dynamically generated a tool (a Python script) to analyze the data and answer the user's question that show cases the following:

**Dynamic Code Generation**: This tool (the Python script) to analyze the data was not manually written or predetermined by the developer. Instead, the LLM itself created the relevant data exploration, correlation analysis, and machine learning code at runtime.

**Isolated Code Execution**: To ensure security and avoid running untrusted code on the host machine, the Python script was executed inside a Docker container using the `execute_python_code` tool. This container had restricted resource access (e.g., no network and limited filesystem access), minimizing potential risks posed by arbitrary code execution.


### Conclusion

The Cookbook provides a guide for developing a custom code interpreter tailored to specific application needs, addressing limitations found in vendor-provided solutions such as language constraints, cost considerations, and the need for flexibility with different LLMs or models.

To recap, the three steps to build an Agentic Application with Dynamic Tool Calling are:
1. Set up an isolated code execution container environment
2. Define and Test the Agents
3. Set up Agentic Orchestration to run the application 

We discussed the importance of isolating the code execution environment to ensure security and avoid running untrusted code on the host machine. With the use case of a CSV file, we demonstrated how to dynamically generate a tool (a Python script) to analyze the data and answer the user's question. We also showed how to execute the code in a Docker container and return the output to the user. 